In [55]:
import json

In [67]:
# function loading json file
def load_data(filepath):
    with open(filepath, 'r') as file:
        return json.load(file)

# checking if it attacks itself
def attacks_itself(argument):
    for i in attacks_relations:
        if i[0] == argument and i[1] == argument:
            attacks_itself = True  
    else:
        attacks_itself = False
    return attacks_itself

# find conflict free sets
def find_cf():
    cf = []
    for key, value in arguments.items():
        if attacks_itself(key):
            continue
        else:
            cf.append({key})
        # find if empty set is applicable
        for attacks in attacks_relations:
            if attacks[1] == key:
                break
        else:
            if {} not in cf:
                cf.append({})
        # add all sets that do not attack each other
        for key2, value2 in arguments.items():
            if key != key2:
                for i in attacks_relations:
                    if (i[1] == key and i[0] == key2) or (i[1] == key2 and i[0] == key):
                        break
                else:
                    if {key2, key} not in cf:
                        cf.append({key2, key})
    return cf

# I also created a function for stable, not 100% if I understood the definition of stable correctly though
# but works on the provided examples
def find_stb():
    cf = find_cf()
    stb = cf.copy()
    for set in cf:
        if len(set) <= 1:
            stb.remove(set)
            continue
        for key, value in arguments.items():
            if key not in set:
                for item in set:
                    if [item, key] in attacks_relations:
                        break
                else:
                    stb.remove(set)
                    break
    return stb

# you can use this code example why we wanted to take the other one 
# as this one is longer 1. seeing which arguments are not admissable and then
# remove them from the conflict free list compared to go through each set in cf 
# and directly remove it if it does not defend itself

# Could imagine thogh that the runtime is slightly faster in the first example as smaller loops

def find_adm():
    cf = find_cf()
    adm = cf.copy()
    
    not_adm = []
    
    for key, value in arguments.items():
        outs = 0
        ins = 0
        for i in attacks_relations:
                if i[0] == key:
                    outs += 1
                if i[1] == key:
                    ins += 1
        if ins > outs:
            not_adm.append(key)
    print(not_adm)
 
    for set in adm[:]:
        if set == {}:
            adm.remove(set)
            continue
        for element in not_adm:
            if (element in set) and (set in adm):
                adm.remove(set)
    return adm

# finds admissable arguments different way of coding
def find_adm2():
    cf = find_cf()
    adm2 = cf.copy()
    
    for set in adm2[:]:
        if set == {}:
            adm2.remove(set)
            continue
        for element in set:
            ins = 0
            outs = 0
            for i in attacks_relations:
                if i[0] == element:
                    outs += 1
                if i[1] == element:
                    ins += 1
            if ins > outs:
                adm2.remove(set)
                break
    return adm2
    
def proof_ca_adm(argument):
    adm = find_adm2()
    is_ca = False
    for i in adm:
        if argument in i:
            is_ca = True
            break
    return is_ca

def proof_ca_stb(argument):
    stb = find_stb()
    is_ca = False
    for i in stb:
        if argument in i:
            is_ca = True
            break
    return is_ca
                

In [68]:
# needs to be input later on but I guess it is easier if we just do it without input for testing purposes
data = load_data("example-input-format-2.json")
arguments = data["Arguments"]
attacks_relations = data["Attack Relations"]

# needs to be input later on but I guess it is easier if we just do it without input for testing purposes
argument= "a"

if proof_ca_adm(argument):
    print("Your argument is credisously acceptable with regards to admissable")
else:
    print("Your argument is not credisously acceptable with regards to admissable")

if proof_ca_stb(argument):
    print("Your argument is credisously acceptable with regards to stable")
else:
    print("Your argument is not credisously acceptable with regards to stable")


Your argument is credisously acceptable with regards to admissable
Your argument is not credisously acceptable with regards to stable


In [ ]:
# optimized code form chatGPT
def load_data(filepath):
    with open(filepath, 'r') as file:
        return json.load(file)

def attacks_itself(argument):
    return (argument, argument) in attacks_relations_set

def find_cf():
    cf = [{}]
    for key in arguments:
        if not attacks_itself(key):
            cf.append({key})
            for other_key in arguments:
                if key != other_key and (other_key, key) not in attacks_relations_set and (key, other_key) not in attacks_relations_set:
                    cf.append({key, other_key})
    return cf

def find_stb():
    cf = find_cf()
    return [s for s in cf if len(s) > 1 and all(not (item, key) in attacks_relations_set for key in arguments for item in s if key not in s)]

def find_adm():
    cf = find_cf()
    return [s for s in cf if s and all(ins_outs[key][0] <= ins_outs[key][1] for key in s)]

def proof_ca_adm(argument):
    return any(argument in s for s in find_adm())

def proof_ca_stb(argument):
    return any(argument in s for s in find_stb())

# Preprocessing
attacks_relations_set = set(attacks_relations)
ins_outs = {key: (sum(i[0] == key for i in attacks_relations), sum(i[1] == key for i in attacks_relations)) for key in arguments}